In [6]:
input_word2vec = input_word2vec(s, word2vec)
#print(input_word2vec)

In [69]:
print(s)

[',,镜子,,,']


In [62]:
print(data_dict[('暧昧', '感情像牛奶一杯,越甜越让人生畏')])

[['像', '让'], [], ['感情', '牛奶', '人生'], [], ['越甜'], []]


In [39]:
# -*- coding: utf-8 -*-
import os
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import csv
import jieba.posseg as pseg
import datetime

st_len = 6


def load_word2vec(file_name):
    word2vec = {}
    with open(file_name,'r') as f:
        s = f.readlines()
        for line in s:
            line = line[1:]
            line = line.split(" ")
            word2vec[line[0]] = line[1:-1]
    return word2vec
"""
将时间,地点,名词,动词存好
"""
def identify_word(words_list):
    verb = []
    place = []
    noun = []
    adj = []
    nr = []
    time = []
    #print(words_list)
    for w in words_list:
        w = w.split("(")
        word = w[0]
        #print(w)
        flag = w[1][:-1]
        if flag == 'v':
            verb.append(word)
        if flag == 'ns':
            place.append(word)
        if flag == 'n':
            noun.append(word)
        if flag == 'a':
            adj.append(word)
        if flag == 'nr':
            nr.append(word)
        if flag == 't':
            time.append(word)
    return [verb,place, noun,adj,nr,time]


def process_file(file_dir):
    data_dict = {}
    for root,dirs,files in os.walk(file_dir):
        pass
    #print(files)
    for file in files[:]:
        #print(file)
        with open(file_dir + "//" + file,"r") as f:
            lines = f.readlines()
            #print(lines)
            for i in range(0,len(lines),2):
                #print(i)
                #截取原歌词
                sentence = lines[i][0:-2]
                #print(file + "-----"+sentence)
                #截取处理好的句子
                words_list = lines[i + 1][:-2].split(",")
                if len(words_list) > 1:
                    data_dict[(file,sentence)] = identify_word(words_list)
                #print(data_dict)
    return data_dict

def data_word2vec(data_dict,word2vec):
    data_word2vec = {}
    for keys in data_dict.keys():
        data = []
        for x in data_dict[keys]:
            word_vec_pair = {}
            if len(x) > 0:
                for word in x:
                    if word2vec.get(word) != None:
                        word_vec_pair[word] = word2vec[word]
                    else:
                        l = list(word)
                        for elem in l:
                            if word2vec.get(elem) != None:
                                word_vec_pair[elem] = word2vec[elem]
            data.append(word_vec_pair)
        data_word2vec[keys] = data
    return data_word2vec
"""
输入的是以逗号间隔的时间,地点,名词,动词
将它们转换为词向量,如果有多个同类的(如同为名词),输入时以空格间隔
"""
def input_word2vec(inputs,word2vec):
    #print(inputs)
    input_word2vec = []
    
    for line in inputs:
        #print(line)
        one_input = []
        x = line.split(',')
        #print(x)
        #print(len(x))
        for d in x:
            word_vec_pair = {}
            if len(d) > 0:
                #print(d)
                words = d.split(" ")
                for w in words:
                    #print(w)
                    if word2vec.get(w) != None:
                        word_vec_pair[w] = word2vec[w]
                    else:
                        l = list(w)
                        #print(l)
                        for elem in l:
                            if word2vec.get(elem) != None:
                                word_vec_pair[elem] = word2vec[elem]
            one_input.append(word_vec_pair)
            #print(one_input)
        input_word2vec.append(one_input)
    return input_word2vec

def calculate_similarity(input_word2vec,data_word2vec):
    for inputs in input_word2vec:
        #print(inputs)
        a = [np.array(list(inputs[i].values())) for i in range(st_len)]
        scores = {}
        for key in data_word2vec.keys():
            sentence = data_word2vec[key]
            b = [np.array(list(sentence[i].values())) for i in range(st_len)]
            count = 0
            score = 0
            for i in range(st_len):
                for j in range(st_len):
                    if a[i].shape[0] != 0 and b[j].shape[0] != 0:
                        count += 1.0
                        """
                        cosine_similarity(a[i],b[i])返回一个余弦矩阵,矩阵第一行代表a[i]的第一个元素和b[i]的所有元素的余弦值
                        np.mean(cosine_similarity(a[i],b[i]),axis = 0)
                        每i列代表b中的第i个元素和a中的所有元素的余弦值的平均值
                        """
                        score += np.mean(np.mean(cosine_similarity(a[i],b[j]),axis = 0))
                    elif a[i].shape[0] != 0 and b[j].shape[0] == 0:
                        count += 1.0
                        score += 0.1
                    elif a[i].shape[0] == 0 and b[j].shape[0] != 0:
                        count += 1.0
                        score += 0.1
            if count > 0:
                score /= count
            scores[key] = score
        ma = {}
        #将所有分数变成一个列表
        score_values = np.array(list(scores.values()))
        #print(len(score_values))
        #将分数映射到下标
        for i in range(len(score_values)):
            s = score_values[i]
            ma[i] = s
            
        #将所有键变成一个列表
        score_keys = list(scores.keys())
        #print(len(score_keys))
        #排序
        sorted_score_values = sorted(ma.items(),key = lambda item:item[1],reverse = True)
        #print(len(sorted_score_values))
        #print(sorted_score_values)
        #取前10个最大的值对应的键
        s_len = 10
        cnt = 0
        res = []
        for i in range(len(score_values)):
            idx = sorted_score_values[i][0]
            ans = score_keys[idx]
            res.append(ans)
            cnt += 1
            if cnt > s_len:
                break
        print("=*="*16)
        print("匹配的歌词及推荐歌曲如下:")
        for i in range(s_len):
            print(res[i],"分数:",sorted_score_values[i][1])
        #取最大值对应的下标
        #print(score_keys[np.argmax(score_values)])

def pp(res,x):
    if len(x) != 0:
        s = ' '.join(x)
        res.append(s)
        res.append(',')
    else:
        s = ','
        res.append(s)
    

def process(sentences):
    print(sentences)
    ans = []
    #[verb,place, noun,adj,nr,time]
    for words in sentences:
        words_cut = pseg.cut(words)
        verb = []
        place = []
        noun = []
        adj = []
        nr = []
        time = []
        for word,flag in words_cut:
            if flag == 'v':
                verb.append(word)
            if flag == 'ns':
                place.append(word)
            if flag == 'n':
                noun.append(word)
            if flag == 'a':
                adj.append(word)
            if flag == 'nr':
                nr.append(word)
            if flag == 't':
                time.append(word)
     #   print(verb,place,noun,adj,nr,time)
        res = []
        pp(res,verb)
        pp(res,place)
        pp(res,noun)
        pp(res,adj)
        pp(res,nr)
        pp(res,time)
        r = ''.join(res)
    ans.append(r[:-1])
    return ans        
if __name__ == '__main__':
    starttime = datetime.datetime.now()
    
    s = ['美丽的泡沫']
    s = process(s)
    print(s)
    word2vec = load_word2vec("emb")
    data_dict = process_file("test")
    #print(data_dict[''])
    data_word2vec = data_word2vec(data_dict, word2vec)
    input_word2vec = input_word2vec(s, word2vec)
#   print(len(input_word2vec))
    #print(input_word2vec)
    #print(data_word2vec[('爱很美', '痛苦过一切不会好的')])
    calculate_similarity(input_word2vec, data_word2vec)
    
    endtime = datetime.datetime.now()
    print (endtime - starttime)


['美丽的泡沫']
[',美丽,泡沫,,,']
=*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*=
匹配的歌词及推荐歌曲如下:
('丁香花', '开满紫色美丽的鲜花') 分数: 0.2265940768570183
('稀客', '难道我美丽年华,为装饰一下,夜里你那无聊的牵挂') 分数: 0.22505642723280964
('读者', '叶落时秋夜深了,当美丽的梦醒了') 分数: 0.22350839711702583
('天高地厚', '想拥抱在最美丽的那一刻') 分数: 0.2231444299070813
('叶子', '美丽的女孩叫叶子她经常这么说') 分数: 0.2186620570792943
('泡沫', '美丽的泡沫,虽然一刹花火,你所有承诺,虽然都太脆弱') 分数: 0.21857125473602398
('涩', '在美丽的花丛中自由的穿行') 分数: 0.21743568030453472
('一如年少模样', '刻骨于雪,失落于风,长情意在,夜雨香') 分数: 0.21523613388365287
('败将', '是谁享受被爱,虚荣的美丽') 分数: 0.21499921110993636
('枫', '清晰透明,就像美丽的风景') 分数: 0.21402754805206475
0:01:43.720331


In [12]:
sentence = '美丽的风景'
sen_cut = pseg.cut(sentence)
for word,flag in sen_cut:
    print(word,flag)

美丽 ns
的 uj
风景 n


In [35]:
print(input_word2vec)

[[{}, {'美丽': ['0.212634', '-0.173637', '-0.021092', '0.191623', '-0.174932', '-0.268263', '0.057935', '-0.354008', '0.207358', '-0.246444', '-0.223445', '-0.072489', '-0.125879', '0.027127', '-0.034760', '-0.176511', '0.055988', '0.041325', '0.295452', '0.009584', '-0.233390', '0.128507', '0.037568', '-0.218624', '-0.034388', '-0.141421', '-0.057953', '0.015135', '0.208888', '0.042806', '0.027278', '0.270158', '-0.166847', '-0.280793', '0.276585', '-0.292066', '0.263969', '0.210527', '0.173650', '0.118095', '-0.053519', '0.061627', '0.236300', '-0.276772', '0.032327', '0.062297', '0.351636', '-0.181200', '0.206530', '0.212210', '0.238894', '-0.011141', '-0.102566', '-0.040163', '0.098729', '0.111934', '0.286586', '0.032763', '0.408281', '-0.100791', '-0.284800', '0.168941', '0.212889', '0.096645', '-0.176822', '0.285981', '-0.270171', '0.023263', '0.013848', '-0.102203', '0.021113', '0.081863', '-0.205557', '0.064504', '-0.086368', '0.321344', '-0.319885', '0.203529', '-0.130109', '0.0

In [14]:
print(data_word2vec[(('如果我变成回忆', '越美丽越让我忐忑'))])

[{'让': ['0.143540', '0.026234', '-0.386745', '0.012148', '0.035704', '-0.055825', '0.312892', '-0.022235', '0.178201', '-0.079659', '-0.023059', '0.014054', '-0.050659', '-0.000425', '-0.146962', '-0.093942', '-0.031436', '-0.066887', '-0.027163', '0.241255', '0.028988', '-0.062302', '0.061601', '-0.075105', '-0.171692', '-0.066024', '-0.118390', '0.344424', '0.066700', '0.005007', '0.052707', '0.059668', '-0.030846', '-0.112460', '0.027435', '-0.274016', '-0.085332', '0.027250', '-0.053591', '0.188849', '-0.201258', '0.137318', '0.000649', '-0.368385', '-0.003455', '-0.109074', '0.307439', '-0.127029', '0.116597', '0.140283', '-0.108308', '0.128349', '-0.205543', '-0.066517', '0.199359', '-0.146825', '0.159245', '0.350113', '-0.076850', '0.082493', '-0.058641', '0.113937', '-0.085886', '-0.017807', '-0.010023', '-0.008921', '0.072202', '0.093035', '0.249034', '-0.027446', '-0.106880', '-0.196174', '-0.175535', '0.030169', '-0.070369', '-0.292246', '0.023837', '-0.055177', '-0.120705',

In [36]:
x = data_word2vec[('十字路口', '美丽的洁白的雪呀')]
y = input_word2vec[0]
print(x == y)

False


In [16]:
print(data_dict.keys())

dict_keys([('如果我变成回忆', '累了,照惯例努力清醒着'), ('如果我变成回忆', '也照惯例想你了'), ('如果我变成回忆', '好怕一放心睡了'), ('如果我变成回忆', '心跳在梦中,不听话的,就停止了'), ('如果我变成回忆', '听着,呼吸像浪潮拍动着'), ('如果我变成回忆', '越美丽越让我忐忑'), ('如果我变成回忆', '我还能珍惜什么'), ('如果我变成回忆', '如果我连自己的脉搏,都难掌握'), ('如果我变成回忆', '如果我变成回忆,退出了这场生命'), ('如果我变成回忆', '留下你错愕哭泣'), ('如果我变成回忆', '我冰冷身体,拥抱不了你'), ('如果我变成回忆', '想到我让深爱的你,人海孤独旅行'), ('如果我变成回忆', '我会恨自己,如此狠心'), ('如果我变成回忆', '如果我变成回忆,终于没那么幸运'), ('如果我变成回忆', '没机会白着头发,蹒跚牵着你,看晚霞落尽'), ('如果我变成回忆', '漫长时光总有一天你会伤心痊愈'), ('如果我变成回忆', '若有人可以,让他陪你,我不怪你'), ('如果我变成回忆', '快乐,什么时候会结束呢'), ('如果我变成回忆', '哪一刻是最后一刻'), ('如果我变成回忆', '想把你紧紧抱着'), ('如果我变成回忆', '可知你是我生命中的,最舍不得'), ('如果我变成回忆', '若有人可以,让他陪你'), ('如果我变成回忆', '如果我变成回忆,最怕我太不争气'), ('如果我变成回忆', '顽固的赖在空气,霸占你心里,每一寸缝隙'), ('如果我变成回忆', '连累依然爱我的你,痛苦承受失去'), ('如果我变成回忆', '这样不公平,请你尽力,把我忘记'), ('了不起寂寞', '我还没解开,这一题'), ('了不起寂寞', '为什么骗自己'), ('了不起寂寞', '总说,不在意'), ('了不起寂寞', '跟真实的我,有差距'), ('了不起寂寞', '还不擅长,控制情绪'), ('了不起寂寞', '谁说不爱了,走开就行'), ('了不起寂寞', '我的确迟钝'), ('了不起寂寞', '连恨都太小心'), ('了不起寂寞', '微笑太久了'), ('了不起寂寞', '不自觉快要窒息'), ('了不起寂寞', 

In [37]:
a = [np.array(list(x[i].values())) for i in range(st_len)]
b = [np.array(list(y[i].values())) for i in range(st_len)]
count = 0
score = 0
#print(a)
#print(b)
for i in range(st_len):

    if a[i].shape[0] != 0 and b[i].shape[0] != 0:
        print(i)
        count += 1.0
        """
        cosine_similarity(a[i],b[i])返回一个余弦矩阵,矩阵第一行代表a[i]的第一个元素和b[i]的所有元素的余弦值
        np.mean(cosine_similarity(a[i],b[i]),axis = 0)
        每i列代表b中的第i个元素和a中的所有元素的余弦值的平均值
        """
        score += np.mean(np.mean(cosine_similarity(a[i],b[i]),axis = 0))
        #print(a[i],b[i])
        print(score)
    
if count > 0 :
    score /= count
#print(score)
# if a == b:
#     print(True)
# else:
#     print(False)
print(a[4].shape[0])
print(b[4].shape[0])

1
1.0
2
1.627637444257996
0
0
